# Tutorial part 4

In [part 3](part3.ipynb) we have learnt about portfolio states. We will continue with that, here.

## Example data

We start with a similar portfolio state as in the previous part:

In [3]:
import portfolyo as pf
import pandas as pd

index = pd.date_range('2024-09-20', '2024-12-10', freq='15T', inclusive='left', tz='Europe/Berlin')
# Creating offtake portfolio line.
ts_offtake = -1 * pf.dev.w_offtake(index, avg=50)
offtake = pf.PfLine({'w': ts_offtake})
# Creating portfolio line with market prices (here: forward price curve).
ts_prices = pf.dev.p_marketprices(index, avg=200)
prices = pf.PfLine({'p': ts_prices})

# Creating portfolio line with sourced volume.
ts_sourced_power1, ts_sourced_price1 = pf.dev.wp_sourced(ts_offtake, 'QS', 0.3, p_avg=120)
sourced_quarters = pf.PfLine({'w': ts_sourced_power1, 'p': ts_sourced_price1})
ts_sourced_power2, ts_sourced_price2 = pf.dev.wp_sourced(ts_offtake, 'MS', 0.2, p_avg=150)
sourced_months = pf.PfLine({'w': ts_sourced_power2, 'p': ts_sourced_price2})
sourced = pf.MultiPfLine({'quarter_products': sourced_quarters, 'month_products': sourced_months})

# Create the portfolio state.
ref = pf.PfState(offtake, prices, sourced)

### Quick recap

The current best-guess procurement price is the following:

In [10]:
ref.asfreq("MS").pnl_cost.p # or ref.pnl_cost.asfreq("MS").p

c:\Users\ruud.wijtvliet\Anaconda3\envs\pf38\lib\site-packages\pint_pandas\pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


2024-10-01 00:00:00+02:00    166.33022586942107
2024-11-01 00:00:00+01:00      164.470891672878
Freq: MS, Name: p, dtype: pint[Eur/MWh]

Our portfolio is only partly hedged...

In [11]:
ref.asfreq("MS").sourcedfraction

c:\Users\ruud.wijtvliet\Anaconda3\envs\pf38\lib\site-packages\pint_pandas\pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


2024-10-01 00:00:00+02:00    0.46655604454524774
2024-11-01 00:00:00+01:00     0.5526316876601053
Freq: MS, Name: fraction, dtype: pint[]

... which means that a market price increase has a significant impact on the procurement price:

In [12]:
higherprices = prices + pf.Q_(50, "Eur/MWh")
pfs_higherprices = ref.set_unsourcedprice(higherprices)
pfs_higherprices.asfreq("MS").pnl_cost.p

c:\Users\ruud.wijtvliet\Anaconda3\envs\pf38\lib\site-packages\pint_pandas\pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


2024-10-01 00:00:00+02:00    193.00242364215893
2024-11-01 00:00:00+01:00     186.8393072898724
Freq: MS, Name: p, dtype: pint[Eur/MWh]

## Hedging

Hedging can reduce the sensitivity of our portfolio to changes in the market price. Given the current market price curve, we can calculate how much we'd need to source to obtain a fully hedged portfolio:

In [15]:
needed = ref.hedge_of_unsourced("vol", "MS")  # volume hedge with month products

c:\users\ruud.wijtvliet\ruud\python\dev\portfolyo\portfolyo\tools\stamps.py:404: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  freq = ts_left.freq


Let's say we procure exactly that volume. We can add it to the sourced volume in our portfolio state:

In [18]:
pfs_hedged = ref.add_sourced({"newvolume": needed})
pfs_hedged

KeyError: "Found item with unexpected key/name 'newvolume'. Should be one of w, q, p, r, nodim, agn."